In [18]:
import torch
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import torchvision
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torch import nn
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore") 
import sys
sys.path.append('../scripts/')
from metrics import find_metrics, find_metrics_macro

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
csv_file = pd.read_csv('../../files/train.csv')
csv_file['updated_paths'] = csv_file['image'].apply(lambda x: '../../files/train_images/' + x)

In [4]:
def split_datasets(csv_file, test_size):
    train, test = train_test_split(csv_file, test_size=test_size, random_state=42)
    train, val = train_test_split(train, test_size=test_size, random_state=42)
    return train, val, test

In [5]:
def encoding_data(csv_data):
    labels = csv_data['species'].values.tolist()
    encoder = OneHotEncoder()
    encoder = encoder.fit(np.array(labels).reshape(-1, 1))
    return encoder

In [6]:
encoder = encoding_data(csv_file)

In [7]:
class DolphinDataset(Dataset):
    def __init__(self, csv_file, encoder, return_label = True):
        self.csv_file = csv_file
        self.images = csv_file['updated_paths'].values.tolist()
        self.labels = csv_file['species'].values.tolist()
        self.encoder = encoder
        self.encoded_labels = self.encoder.transform(np.array(self.labels).reshape(-1, 1)).toarray()
        self.convert_rgb = torchvision.transforms.Lambda(lambda x: x.repeat(3, 1, 1))
        self.return_label = return_label

    def __len__(self):
        return len(self.csv_file)

    def __getitem__(self, idx):
        img = torchvision.io.read_file(self.images[idx])
        img = torchvision.io.decode_jpeg(img)
        if img.shape[0] != 3:
            img = self.convert_rgb(img)
        img = torchvision.transforms.functional.resize(img, (512, 512))
        img = img / 255.0
        if self.return_label:
            return img, torch.Tensor(self.encoded_labels[idx])
        else:
            return img

In [8]:
train, val, test = split_datasets(csv_file, test_size = 0.01)

In [9]:
def get_model(load_weights = None):
    incep_model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained = False)
    incep_model.fc = nn.Linear(2048, 30)
    if load_weights != None:
        incep_model.load_state_dict(torch.load(load_weights)['model_state_dict'])
    return incep_model

In [10]:
categorical_cross_entropy = nn.CrossEntropyLoss()

In [16]:
#  prec, rec = find_metrics_macro(outputs, labels, batch_size=batch_size)
# Appply batch size as well

In [19]:
def eval_model(model, train_dataset, val_dataset):
    model = model.to(device)
    train_loss, val_loss, train_prec, train_rec, val_prec, val_rec = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
    for img, labels in tqdm(train_dataset):
        img = img.to(device)
        labels = labels.to(device)
        outputs = model(img)[0]
        loss = categorical_cross_entropy(outputs, labels)
        prec, rec = find_metrics_macro(outputs, labels, batch_size=64)
        train_loss += loss.item()
        train_prec += prec.item()
        train_rec += rec.item()
    for img, labels in tqdm(val_dataset):
        img = img.to(device)
        labels = labels.to(device)
        outputs = model(img)[0]
        loss = categorical_cross_entropy(outputs, labels)
        prec, rec = find_metrics_macro(outputs, labels, batch_size=64)
        val_loss += loss.item()
        val_prec += prec.item()
        val_rec += rec.item()
    print(f'Train Loss: {train_loss / len(train_dataset)}')
    print(f'Val Loss: {val_loss / len(val_dataset)}')
    print(f'Train Prec: {train_prec / len(train_dataset)}')
    print(f'Val Prec: {val_prec / len(val_dataset)}')
    print(f'Train Rec: {train_rec / len(train_dataset)}')
    print(f'Val Rec: {val_rec / len(val_dataset)}')

In [20]:
# Gether datasets
batch_size = 64
train_dataset = DolphinDataset(train, encoder)
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = False, num_workers = 24, prefetch_factor=2)
val_dataset = DolphinDataset(val, encoder)
val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False, num_workers = 24, prefetch_factor=2)

In [21]:
model = get_model('../../weights/torch_with_momentum_v3/143.pth')
eval_model(model, train_dataloader, val_dataloader)

Using cache found in /home/ubuntu/.cache/torch/hub/pytorch_vision_v0.10.0
100%|██████████| 8/8 [00:07<00:00,  1.00it/s]


Train Loss: 0.6509817068457908
Val Loss: 0.6504473760724068
Train Prec: 0.3607090634038991
Val Prec: 0.37519148737192154
Train Rec: 0.3549150708310135
Val Rec: 0.37114330008625984


In [22]:
model = get_model('../../weights/torch_with_momentum_v3/144.pth')
eval_model(model, train_dataloader, val_dataloader)

Using cache found in /home/ubuntu/.cache/torch/hub/pytorch_vision_v0.10.0
100%|██████████| 8/8 [00:08<00:00,  1.02s/it]

Train Loss: 0.6406922279035344
Val Loss: 0.6469489745795727
Train Prec: 0.3640161359020511
Val Prec: 0.38373006880283356
Train Rec: 0.3585598710590921
Val Rec: 0.3687208369374275


In [23]:
model = get_model('../../weights/torch_with_momentum_v3/145.pth')
eval_model(model, train_dataloader, val_dataloader)

Using cache found in /home/ubuntu/.cache/torch/hub/pytorch_vision_v0.10.0
100%|██████████| 8/8 [00:07<00:00,  1.03it/s]


Train Loss: 0.6372974237898732
Val Loss: 0.6491555720567703
Train Prec: 0.3631360935204474
Val Prec: 0.36930273100733757
Train Rec: 0.3587030590609516
Val Rec: 0.3627403788268566
